In [1]:
from datasets import load_dataset, load_metric, load_from_disk, Audio
from datasets import ClassLabel, DatasetDict, Dataset
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer
from evaluate import load
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import argparse
import json
import logging
import os
import subprocess
import tarfile
import urllib.request

Dataset

In [37]:
magic_data = DatasetDict()

magic_data["train"] = load_dataset('csv', data_files='train.csv')['train']
magic_data["validation"] = load_dataset('csv', data_files='dev.csv')['train']
magic_data["test"] = load_dataset('csv', data_files='test.csv')['train']

magic_data

Using custom data configuration default-ff92f78b727e568e
Found cached dataset csv (/home/ujan/.cache/huggingface/datasets/csv/default-ff92f78b727e568e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-b2626e7933d3dc31
Found cached dataset csv (/home/ujan/.cache/huggingface/datasets/csv/default-b2626e7933d3dc31/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-5d387fce4dcc8507
Found cached dataset csv (/home/ujan/.cache/huggingface/datasets/csv/default-5d387fce4dcc8507/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio_path', 'transcript', 'duration'],
        num_rows: 152903
    })
    validation: Dataset({
        features: ['audio_path', 'transcript', 'duration'],
        num_rows: 9847
    })
    test: Dataset({
        features: ['audio_path', 'transcript', 'duration'],
        num_rows: 21317
    })
})

In [38]:
dataset = magic_data.rename_column('audio_path', 'audio')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))  # downsample to 16k
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 152903
    })
    validation: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 9847
    })
    test: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 21317
    })
})

In [40]:
#dataset = dataset.filter(lambda example: example['audio'] is not None)
dataset = dataset.filter(lambda example: example['transcript'] is not None)
dataset

  0%|          | 0/153 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 152903
    })
    validation: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 9847
    })
    test: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 21317
    })
})

Split into train test

In [46]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset["validation"].remove_columns(["duration", "audio"]))

,transcript
0,但是说你偷别人东西这也是一个非常恶劣的行为也应该是我们要去
1,性能是很强但是
2,每个人的想法都不一样
3,这些问题的话
4,那那你觉得它是它这个负面评价你觉得它那个它那个它那个观点是不是正确的呢
5,对外卖种类其实也挺多的
6,邓邓小平邓小平同志呢还是坚持了咱们这个
7,他的字有些媚俗质感但是他的书法作品确实还是很不错的
8,在咱们中国来说是很重要的一个存在吧
9,仍然活着啊嗯也也是咱们这个


Get vocab

In [47]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'  ######## chinese?

def remove_special_characters(batch):
    batch["transcript"] = re.sub(chars_to_ignore_regex, '', batch["transcript"]).lower()
    return batch

dataset = dataset.map(remove_special_characters)

  0%|          | 0/152903 [00:00<?, ?ex/s]

  0%|          | 0/9847 [00:00<?, ?ex/s]

  0%|          | 0/21317 [00:00<?, ?ex/s]

In [48]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcript"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.

In [49]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'沌': 0,
 '矶': 1,
 '真': 2,
 '筷': 3,
 '疵': 4,
 '垒': 5,
 '褥': 6,
 '胥': 7,
 '翁': 8,
 '孵': 9,
 '腮': 10,
 '歇': 11,
 '道': 12,
 '众': 13,
 '符': 14,
 '瘴': 15,
 '恒': 16,
 '直': 17,
 '巡': 18,
 '灵': 19,
 '鹦': 20,
 '戚': 21,
 '悔': 22,
 '盹': 23,
 '朴': 24,
 '幅': 25,
 '叫': 26,
 '遗': 27,
 '厄': 28,
 '寅': 29,
 '嵩': 30,
 '胖': 31,
 '菲': 32,
 '跶': 33,
 '呈': 34,
 '体': 35,
 '绸': 36,
 '歉': 37,
 '倭': 38,
 '届': 39,
 '龌': 40,
 '俗': 41,
 '尊': 42,
 '乞': 43,
 '瘩': 44,
 '浅': 45,
 '鼎': 46,
 '冠': 47,
 '游': 48,
 '胎': 49,
 '畸': 50,
 '屌': 51,
 '睁': 52,
 '挤': 53,
 '殊': 54,
 '倒': 55,
 '昨': 56,
 '键': 57,
 '麟': 58,
 '立': 59,
 '兄': 60,
 '材': 61,
 '粒': 62,
 '腻': 63,
 '人': 64,
 '提': 65,
 '票': 66,
 '梭': 67,
 '俊': 68,
 '伞': 69,
 '厥': 70,
 '玫': 71,
 '撼': 72,
 '葫': 73,
 '荡': 74,
 '称': 75,
 '贫': 76,
 '兰': 77,
 '猛': 78,
 '番': 79,
 '鞋': 80,
 '烟': 81,
 '膑': 82,
 '幺': 83,
 '嘞': 84,
 '鹅': 85,
 '牌': 86,
 '惫': 87,
 '噻': 88,
 '秘': 89,
 '央': 90,
 '惜': 91,
 '络': 92,
 '圣': 93,
 '喜': 94,
 '伍': 95,
 '唏': 96,
 '则': 97,
 '锢': 98,
 '隔': 99,
 '傣': 100,

Wav2Vec2 feature extractor

In [50]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

4085


To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

In [51]:
with open('magic_data_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [2]:
tokenizer = Wav2Vec2CTCTokenizer("./magic_data_vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [3]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [4]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [60]:
rand_int = random.randint(0, len(dataset['train'])-1)

print(dataset['train'][rand_int]["transcript"])
ipd.Audio(data=dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=16000)

中美贸易关系


In [61]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids
    return batch

In [62]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

#1:   0%|          | 0/38226 [00:00<?, ?ex/s]

#2:   0%|          | 0/38226 [00:00<?, ?ex/s]

#3:   0%|          | 0/38225 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#0:   0%|          | 0/38226 [00:00<?, ?ex/s]

...

#1:   0%|          | 0/5329 [00:00<?, ?ex/s]

#3:   0%|          | 0/5329 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


#2:   0%|          | 0/5329 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your l

In [63]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 152903
    })
    validation: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 9847
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 21317
    })
})

In [64]:
dataset.save_to_disk('magic_data_vectorized')

Saving the dataset (0/53 shards):   0%|          | 0/152903 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/9847 [00:00<?, ? examples/s]

Saving the dataset (0/8 shards):   0%|          | 0/21317 [00:00<?, ? examples/s]

In [5]:
dataset = load_from_disk('magic_data_vectorized')

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 152903
    })
    validation: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 9847
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 21317
    })
})

In [7]:
demo = DatasetDict()
demo['train'] = dataset['train'].select([i for i in range(10000)])
demo['val'] = dataset['validation'].select([i for i in range(1000)])

demo

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 10000
    })
    val: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1000
    })
})

Training & Evaluation

In [8]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [9]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [10]:
#wer = load("wer")
cer_metric = load("/home/ujan/Downloads/evaluate/metrics/cer/cer.py")

In [11]:
model = Wav2Vec2ForCTC.from_pretrained(
    "/home/ujan/Downloads/aishell1_finetune/mandarin-wav2vec2",
    attention_dropout=0.1,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/ujan/Downloads/aishell1_finetune/mandarin-wav2vec2 and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([4233, 768]) in the checkpoint and torch.Size([4085, 768]) in the model instantiated
- lm_head.bias: found shape torch.Size([4233]) in the checkpoint and torch.Size([4085]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore 

In [12]:
model.freeze_feature_encoder()

In [13]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    #wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [14]:
training_args = TrainingArguments(
    output_dir='/home/ujan/Notebooks/aishell_ctc',
    group_by_length=True,
    #per_device_train_batch_size=32,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_accumulation_steps=16,
    num_train_epochs=30,
    fp16=True,
    gradient_checkpointing=True, 
    save_steps=500, #
    eval_steps=500, #
    logging_steps=500, #
    learning_rate=0.0002,
    weight_decay=0.005,
    warmup_steps=500,
    save_total_limit=2,
    dataloader_num_workers=0,
)

In [15]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=demo["train"],
    eval_dataset=demo["val"], # validation
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18750
  Number of trainable parameters = 93312629
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warn

Step,Training Loss,Validation Loss,Cer
500,21.543500,7.848636,1.000000
1000,6.140400,9.108735,1.000000
1500,6.106300,7.989006,1.000000
2000,6.110100,9.915636,1.000000
2500,6.101700,9.542008,1.000000
3000,6.094900,8.079228,1.000000
3500,6.093000,10.028331,1.000000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/aishell_ctc/checkpoint-500
Configuration saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/config.json
Model weights saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/preprocessor_config.json
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/aishell_ctc/checkpoint-1000
Configuration saved in /home/ujan/Notebooks/aishell_